In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [2]:
dataset=pd.read_csv('Churn_Modelling.csv')
dataset.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [3]:
dataset=dataset.drop(['RowNumber','CustomerId','Surname'],axis=1)
LabelEncoder_gender=LabelEncoder()
dataset['Gender']=LabelEncoder_gender.fit_transform(dataset['Gender'])

In [3]:
onehotencoder_geography=OneHotEncoder()
geo_encoder=onehotencoder_geography.fit_transform(dataset[['Geography']])

In [7]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
onehotencoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
#Converted into DF
coded_df=pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder_geography.get_feature_names_out(['Geography']))
coded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [10]:
dataset=dataset.drop(['Geography'],axis=1)
dataset.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  
0               1        101348.88       1  
1               1        112542.58       0  
2               0        113931.57       1  
3               0         93826.63       0  
4               1         79084.10       0

In [11]:
#Concatinating
dataset=pd.concat([dataset,coded_df],axis=1)
dataset.head()

CreditScore  Gender  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0          619       0   42       2       0.00              1          1   
1          608       0   41       1   83807.86              1          0   
2          502       0   42       8  159660.80              3          1   
3          699       0   39       1       0.00              2          0   
4          850       0   43       2  125510.82              1          1   

   IsActiveMember  EstimatedSalary  Exited  Geography_France  \
0               1        101348.88       1               1.0   
1               1        112542.58       0               0.0   
2               0        113931.57       1               1.0   
3               0         93826.63       0               1.0   
4               1         79084.10       0               0.0   

   Geography_Germany  Geography_Spain  
0                0.0              0.0  
1                0.0              1.0  
2                0.0              0.0  
3                0.0              0.0  
4                0.0              1.0

In [ ]:
#Save The encoders
with open('LabelEncoder_gender.pkl','wb') as file:
    pickle.dump(LabelEncoder_gender,file)

with open ('geo_encoder.pkl','wb') as file:
    pickle.dump(onehotencoder_geography,file)

In [13]:
X=dataset.drop(['Exited'],axis=1)
y=dataset['Exited']

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=42)

In [15]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
with open ('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN IMPLEMENTATION

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [19]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64)                832       
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()


In [25]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])#adam has fixed LR

In [26]:
#Setup TensorFlow
log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [30]:
EarlyStopping_callbacks=EarlyStopping(patience=10,monitor='val_loss',restore_best_weights=True)

In [31]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,EarlyStopping_callbacks]
)

Epoch 1/100
250/250 [==============================] - 0s 455us/step - loss: 0.3430 - accuracy: 0.8601 - val_loss: 0.3409 - val_accuracy: 0.8620
Epoch 2/100
250/250 [==============================] - 0s 390us/step - loss: 0.3395 - accuracy: 0.8612 - val_loss: 0.3392 - val_accuracy: 0.8585
Epoch 3/100
250/250 [==============================] - 0s 391us/step - loss: 0.3366 - accuracy: 0.8610 - val_loss: 0.3374 - val_accuracy: 0.8580
Epoch 4/100
250/250 [==============================] - 0s 397us/step - loss: 0.3333 - accuracy: 0.8634 - val_loss: 0.3345 - val_accuracy: 0.8600
Epoch 5/100
250/250 [==============================] - 0s 387us/step - loss: 0.3318 - accuracy: 0.8622 - val_loss: 0.3684 - val_accuracy: 0.8475
Epoch 6/100
250/250 [==============================] - 0s 478us/step - loss: 0.3314 - accuracy: 0.8612 - val_loss: 0.3420 - val_accuracy: 0.8580
Epoch 7/100
250/250 [==============================] - 0s 400us/step - loss: 0.3279 - accuracy: 0.8654 - val_loss: 0.3537 - val_ac

In [32]:
model.save('model.h5')

/Users/1persentbetter/Documents/Bugs_Villa/Gen AI/Sec_13_Ann Project/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
#Load Tensorboard Extension
%load_ext tensorboard

In [36]:
%tensorboard --logdir logs/fit20250914-162316/

Reusing TensorBoard on port 6006 (pid 7225), started 0:02:11 ago. (Use '!kill 7225' to kill it.)